In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
# You may change the mhealth_activity module but your algorithm must support the original version
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path
from tqdm import tqdm
from multiprocessing import Pool
import pickle
# For interactive plots, uncomment the following line
# %matplotlib widget

In [2]:
create_data_pickle = True
if create_data_pickle:
    files = os.listdir('data/train')
    files.sort()
    print(files)
    list_of_dicts = []
    types_to_include = ['ax', 'ay', 'az','gx', 'gy', 'gz', 'mx', 'my', 'mz', 'speed', 'longitude', 'latitude', 'altitude', 'phone_steps', 'temperature']
    # types_to_include = ['ax', 'ay', 'az','gx', 'gy', 'gz','mx', 'my', 'mz',  'speed', 'altitude', 'phone_steps', 'temperature']

    for file in tqdm(files):
        Dict = {}
        d = Recording(os.path.join('data/train',file))

        Dict['labels'] = d.labels
        for data_type in types_to_include:
            if data_type in d.data.keys():
                Dict[data_type] = d.data[data_type]
        list_of_dicts.append(Dict)

    data = pd.DataFrame(list_of_dicts)
    data.to_pickle(path='data/pickled_and_sorted_training_data.pkl.zst', compression={'method': 'zstd'})
else:
    data = pd.read_pickle('data/pickled_and_sorted_training_data.pkl.zst')
    data = data.drop(columns=['labels', 'ax', 'ay', 'az', 'mx', 'my', 'mz', 'speed', 'altitude',
       'phone_steps', 'temperature'])

['train_trace_000.pkl', 'train_trace_001.pkl', 'train_trace_002.pkl', 'train_trace_003.pkl', 'train_trace_004.pkl', 'train_trace_005.pkl', 'train_trace_006.pkl', 'train_trace_007.pkl', 'train_trace_008.pkl', 'train_trace_009.pkl', 'train_trace_010.pkl', 'train_trace_011.pkl', 'train_trace_012.pkl', 'train_trace_013.pkl', 'train_trace_014.pkl', 'train_trace_015.pkl', 'train_trace_016.pkl', 'train_trace_017.pkl', 'train_trace_018.pkl', 'train_trace_019.pkl', 'train_trace_020.pkl', 'train_trace_021.pkl', 'train_trace_022.pkl', 'train_trace_023.pkl', 'train_trace_024.pkl', 'train_trace_025.pkl', 'train_trace_026.pkl', 'train_trace_027.pkl', 'train_trace_028.pkl', 'train_trace_029.pkl', 'train_trace_030.pkl', 'train_trace_031.pkl', 'train_trace_032.pkl', 'train_trace_033.pkl', 'train_trace_034.pkl', 'train_trace_035.pkl', 'train_trace_036.pkl', 'train_trace_037.pkl', 'train_trace_038.pkl', 'train_trace_039.pkl', 'train_trace_040.pkl', 'train_trace_041.pkl', 'train_trace_042.pkl', 'train_tra

100%|██████████| 396/396 [01:41<00:00,  3.89it/s]


In [5]:
def mag_norm(input):
    return (np.sqrt(input[1]['mx'].values**2 + input[1]['my'].values**2 + input[1]['mz'].values**2))

def calc_magneto_mag(input):

    mag = []
    with Pool(4) as p:
        mag = p.map(mag_norm, input.iterrows())        
    # print(mag)
    return mag

def accel_norm(input):
    return (np.sqrt(input[1]['ax'].values**2 + input[1]['ay'].values**2 + input[1]['az'].values**2))

def calc_accel_mag(input):

    mag = []
    with Pool(4) as p:
        mag = p.map(accel_norm, input.iterrows())        
    # print(mag)
    return mag

def gyro_norm(input):
    return (np.sqrt(input[1]['gx'].values**2 + input[1]['gy'].values**2 + input[1]['gz'].values**2))

def calc_gyro_mag(input):

    mag = []
    with Pool(4) as p:
        mag = p.map(gyro_norm, input.iterrows())        
    # print(mag)
    return mag


In [6]:




calc_watch_vals = True
if calc_watch_vals:

    accel_mag   = calc_accel_mag(data)
    print("calced")
    with open('data/accel_mag_train.pkl', 'wb') as f:
        pickle.dump(accel_mag, f)
        f.close()

    # magneto_mag = calc_magneto_mag(data)
    # print("calced")

    # with open('data/magneto_mag_train.pkl', 'wb') as f:
    #     pickle.dump(magneto_mag, f)
    #     f.close()
        
    # gyro_mag    = calc_gyro_mag(data)
    # with open('data/gyro_mag_train.pkl', 'wb') as f:
    #     pickle.dump(gyro_mag, f)
    #     f.close()




    

    

calced


In [ ]:
#handle phone data too, doesnt really work now
def phone_mag_norm(input):
    return (np.sqrt(input[1]['phone_mx'].values**2 + input[1]['phone_my'].values**2 + input[1]['phone_mz'].values**2))

def calc_phone_magneto_mag(input):

    mag = []
    with Pool(8) as p:
        mag = p.map(phone_mag_norm, data.iterrows())        
    # print(mag)
    return mag

def phone_accel_norm(input):
    if(len(input[1]['phone_ax'].values) != len(input[1]['phone_ay'].values**2) or len (input[1]['phone_ay'].values**2 ) != len(input[1]['phone_az'].values**2) or len (input[1]['phone_ax'].values**2 ) != len(input[1]['phone_az'].values**2)):
        print(f"{len(input[1]['phone_ax'].values)} {len(input[1]['phone_ay'].values**2)} {len(input[1]['phone_az'].values**2)}" )

    return (np.sqrt(input[1]['phone_ax'].values**2 + input[1]['phone_ay'].values**2 + input[1]['phone_az'].values**2))

def calc_phone_accel_mag(input):

    mag = []
    with Pool(4) as p:
        mag = p.map(phone_accel_norm, data.iterrows())        
    print(mag)
    return mag

calc_phone_mags = False
if calc_phone_mags:
    phone_accel_mag = calc_phone_accel_mag(data)
    phone_magneto_mag = calc_phone_magneto_mag(data)
    with open('data/phone_accel_mag_train.pkl', 'wb') as f:
        pickle.dump(phone_accel_mag, f)
        f.close()

    with open('data/phone_magneto_mag_train.pkl', 'wb') as f:
        pickle.dump(phone_magneto_mag, f)
        f.close()

In [3]:
#rinse and repeat for test data
create_data_pickle = True
if create_data_pickle:
    files = os.listdir('data/test')
    files.sort()
    list_of_dicts = []
    types_to_include = ['ax', 'ay', 'az', 'gx', 'gy', 'gz', 'mx', 'my', 'mz']

    for file in tqdm(files):
        Dict = {}
        d = Recording(os.path.join('data/test',file))

        Dict['labels'] = d.labels
        for data_type in types_to_include:
            if data_type in d.data.keys():
                Dict[data_type] = d.data[data_type]
        list_of_dicts.append(Dict)

    data_test = pd.DataFrame(list_of_dicts)
    data_test.to_pickle(path='data/pickled_and_sorted_test_data.pkl.zst', compression={'method': 'zstd'})
else:
    data_test = pd.read_pickle('data/pickled_and_sorted_test_data.pkl.zst')

  0%|          | 0/280 [00:00<?, ?it/s]

100%|██████████| 280/280 [00:19<00:00, 14.64it/s]


In [6]:
calc_watch_vals = True
if calc_watch_vals:

    accel_mag   = calc_accel_mag(data_test)

    with open('data/accel_mag_test.pkl', 'wb') as f:
        pickle.dump(accel_mag, f)
        f.close()

    magneto_mag = calc_magneto_mag(data_test)
    print("calced")

    with open('data/magneto_mag_test.pkl', 'wb') as f:
        pickle.dump(magneto_mag, f)
        f.close()
        
    gyro_mag    = calc_gyro_mag(data_test)
    with open('data/gyro_mag_test.pkl', 'wb') as f:
        pickle.dump(gyro_mag, f)
        f.close()


calced


In [6]:
d = Recording('data/train/train_trace_000.pkl')
print(d.data.keys())

dict_keys(['phone_mx', 'gx', 'altitude', 'phone_rotx', 'phone_steps', 'longitude', 'phone_gravy', 'lostPackets', 'phone_gz', 'latitude', 'phone_my', 'timestamp', 'gz', 'ax', 'my', 'phone_pressure', 'phone_gravx', 'phone_gy', 'phone_orientationx', 'temperature', 'phone_az', 'mz', 'az', 'gy', 'phone_mz', 'phone_rotm', 'phone_ax', 'packetNumber', 'phone_orientationz', 'phone_gravz', 'speed', 'bearing', 'phone_gx', 'phone_lax', 'phone_laz', 'phone_rotz', 'ay', 'phone_roty', 'phone_lay', 'mx', 'phone_ay', 'phone_orientationy'])


In [7]:
files = os.listdir('data/train')
files.sort()
list_of_dicts = []
# types_to_include = ['ax', 'ay', 'az','gx', 'gy', 'gz', 'phone_ax', 'phone_ay', 'phone_az', 'mx', 'my', 'mz', 'phone_mx', 'phone_my', 'phone_mz', 'speed', 'longitude', 'latitude', 'altitude', 'phone_steps', 'temperature']
types_to_include = ['ax', 'gx', 'gy', 'gz','mx', 'my', 'mz',  'speed', 'altitude', 'phone_steps', 'temperature']

for file in tqdm(files):
    Dict = {}
    d = Recording(os.path.join('data/train',file))

    Dict["accel_time"] = d.data['ax'].timestamps
    Dict["gyro_time"] = d.data['gx'].timestamps
    Dict["magneto_time"] = d.data['mx'].timestamps

    list_of_dicts.append(Dict)
    
data = pd.DataFrame(list_of_dicts)
data.to_pickle(path='data/pickled_and_sorted_training_data.pkl.zst', compression={'method': 'zstd'})

  4%|▍         | 17/396 [00:06<02:32,  2.48it/s]


KeyboardInterrupt: 